## Imports

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, Trainer, TrainingArguments, AdamW
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from model_utils import transform_label, truncate, encode_tags, OurDataset, evaluate_model, predict_test

In [5]:
SENTENCES_SIZE = 128
TOKENS_NUMBER = 256

### Export des données augmentées

In [6]:
from numpy import True_
from scripts.doccano_to_camembert import JsonlDoccano as jd
file = jd("""input_data/exported_doccano.jsonl""", camembert=True)
#file.run_export("scripts/data/tokens_v5.parquet")
df = file.df_augmented
df.label = df.label.apply(lambda x: transform_label(x)) # Deleting B- I- tags
df.T

Token indices sequence length is longer than the specified maximum sequence length for this model (794 > 512). Running this sequence through the model will result in indexing errors


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,56235,56236,56237,56238,56239,56240,56241,56242,56243,56244,56245,56246,56247,56248,56249,56250,56251,56252,56253,56254,56255,56256,56257,56258,56259,56260,56261,56262,56263,56264,56265,56266,56267,56268,56269,56270,56271,56272,56273,56274,56275,56276,56277,56278,56279,56280,56281,56282,56283,56284,56285,56286,56287,56288,56289,56290,56291,56292,56293,56294,56295,56296,56297,56298,56299,56300,56301,56302,56303,56304,56305,56306,56307,56308,56309,56310,56311,56312,56313,56314,56315,56316,56317,56318,56319,56320,56321,56322,56323,56324,56325,56326,56327,56328,56329,56330,56331,56332,56333,56334,56335,56336,56337,56338,56339,56340,56341,56342,56343,56344,56345,56346,56347,56348,56349,56350,56351,56352,56353,56354,56355,56356,56357,56358,56359,56360,56361,56362,56363,56364,56365,56366,56367,56368,56369,56370,56371,56372,56373,56374,56375,56376,56377,56378,56379,56380,56381,56382,56383,56384,56385,56386,56387,56388,56389,56390,56391,56392,56393,56394,56395,56396,56397,56398,56399,56400,56401,56402,56403,56404,56405,56406,56407,56408,56409,56410,56411,56412,56413,56414,56415,56416,56417,56418,56419,56420,56421,56422,56423,56424,56425,56426,56427,56428,56429,56430,56431,56432,56433,56434,56435,56436,56437,56438,56439,56440,56441,56442,56443,56444,56445,56446,56447,56448,56449,56450,56451,56452,56453,56454,56455,56456,56457,56458,56459,56460,56461,56462,56463,56464,56465,56466,56467,56468,56469,56470,56471,56472,56473,56474,56475,56476,56477,56478,56479,56480,56481,56482,56483,56484
token,▁Un,▁homme,▁âgé,▁de,▁77,▁ans,▁présentait,▁des,▁signes,▁cliniques,▁de,▁pro,stat,isme,.,▁Le,▁TR,▁not,ait,▁une,▁masse,▁ré,ni,tente,.,▁Le,▁taux,▁de,▁PSA,▁était,▁de,▁,"9,5",▁n,g,/,ml,.,▁L,',E,ER,▁et,▁l,',imagerie,▁par,▁résonance,▁magnétique,▁nucléaire,▁(,IRM,),▁de,▁la,▁prostate,▁mont,raient,▁dans,▁la,▁Z,P,▁droite,",",▁une,▁masse,▁,ky,s,tique,▁à,▁parois,▁épaisse,s,▁de,▁4,cm,▁de,▁diamètre,▁et,▁contenant,▁des,▁végétation,s,▁en,dol,umi,nale,s,.,▁L,',analyse,▁,histo,logique,▁des,▁bio,ps,ies,▁pro,statique,s,▁écho,gu,idées,▁montrait,▁un,▁ad,éno,car,ci,nome,▁duc,tal,▁de,▁score,▁de,▁G,le,as,on,▁7,▁(4,+,3),▁et,▁de,▁contingent,▁tra,bé,culaire,▁pré,dom,in,ant,.,▁L,',analyse,▁immuno,-,histo,logique,▁pour,▁les,▁cyto,k,érat,ines,▁de,▁haut,▁poids,▁moléculaire,▁(,anti,▁C,K,▁90,3),▁était,▁négative,.,▁Le,▁stade,▁clinique,▁était,▁lors,▁du,▁diagnostic,▁:,▁T,3,N,x,M,0,.,▁Un,▁traitement,▁par,▁blocage,▁,andro,génique,▁complet,▁a,▁été,▁indiqué,.,▁Le,▁PSA,▁à,▁6,▁mois,▁était,▁à,"▁0,3",6,▁n,g,/,ml,▁puis,▁avait,▁augmenté,▁progressivement,▁jusqu,',à,▁atteindre,▁3,",",2,▁n,g,/,ml,",",▁3,▁ans,▁après,▁le,▁diagnostic,.,▁Une,▁E,ER,▁de,▁contrôle,▁à,▁3,▁ans,▁a,▁été,▁réalisée,▁et,▁montrait,▁la,▁persistance,▁de,▁cette,▁masse,▁,ky,s,tique,▁mais,▁avec,▁des,...,▁en,▁place,▁associé,▁à,▁une,▁en,té,roc,y,st,oplastie,▁d,',élargissement,▁en,▁raison,▁de,▁l,',hypo,can,ie,▁sévère,.,▁Le,▁patient,▁a,▁continué,auto,-,son,dage,s,.,▁Ils,▁sont,▁un,▁patient,▁de,▁59,▁ans,▁qui,▁s,',est,▁présenté,▁aux,▁urgences,▁à,▁une,▁,hémat,urie,▁macro,scopique,▁spontanée,",",▁indo,lore,▁sans,▁fièvre,▁ni,▁perte,▁de,▁poids,.,L,',examen,▁était,▁normal,

## Prétraitement et tokenization

In [7]:
df = df.groupby('id_phrase').agg(lambda x: list(x)).reset_index()
df.columns = ["id_phrase", "tokens", "ner_tags"]
df.head(3)

,id_phrase,tokens,ner_tags
0,1000,"[▁Un, ▁homme, ▁âgé, ▁de, ▁77, ▁ans, ▁présentai...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,2000,"[▁Mr, ., ▁R, ,, R, ▁âgé, ▁de, ▁53, ▁ans, ▁sans...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,2001,"[▁Le, ▁patient, ▁a, ▁été, ▁revu, ▁1, ▁mois, ▁a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [8]:
df = truncate(df) # Toutes les listes de plus de 128 éléments sont dubdivisées en plusieurs listes

In [9]:
tokens = df["tokens"].to_list()
ner_tags = df["ner_tags"].to_list()

In [10]:
#Train test split
train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens, ner_tags, test_size=0.1)

In [11]:
unique_tags = ['O', 'Dosage', 'Drug', 
'Duration', 'Form', 'Frequency', 'Route', 'Treatment']

tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [12]:
#To keep BIO tags
"""#unique_tags = set(tag for doc in ner_tags for tag in doc)
unique_tags = ['O', 'B-Dosage', 'I-Dosage', 'B-Drug', 'I-Drug', 
'B-Duration', 'I-Duration', 'B-Form', 'I-Form', 'B-Frequency', 
'I-Frequency', 'B-Route', 'I-Route', 'B-Treatment', 'I-Treatment']

tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
"""

"#unique_tags = set(tag for doc in ner_tags for tag in doc)\nunique_tags = ['O', 'B-Dosage', 'I-Dosage', 'B-Drug', 'I-Drug', \n'B-Duration', 'I-Duration', 'B-Form', 'I-Form', 'B-Frequency', \n'I-Frequency', 'B-Route', 'I-Route', 'B-Treatment', 'I-Treatment']\n\ntag2id = {tag: id for id, tag in enumerate(unique_tags)}\nid2tag = {id: tag for tag, id in tag2id.items()}\n"

In [13]:
#Tokenization
tokenizer = CamembertTokenizerFast.from_pretrained('camembert-base')
train_encodings = tokenizer(train_tokens, is_split_into_words=True, return_offsets_mapping=True, padding='max_length',
                            truncation='longest_first',
                            max_length=TOKENS_NUMBER)
val_encodings = tokenizer(val_tokens, is_split_into_words=True, return_offsets_mapping=True, padding='max_length',
                            truncation='longest_first',
                            max_length=TOKENS_NUMBER)

train_labels = encode_tags(train_tags, train_encodings, tag2id)
val_labels = encode_tags(val_tags, val_encodings, tag2id)

In [14]:
train_encodings.pop("offset_mapping") 
val_encodings.pop("offset_mapping")

train_dataset = OurDataset(train_encodings, train_labels)
val_dataset = OurDataset(val_encodings, val_labels)

## Train model

In [22]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [23]:
model = CamembertForTokenClassification.from_pretrained('camembert-base', 
                                                        num_labels=len(unique_tags), 
                                                        id2label=id2tag, 
                                                        label2id=tag2id).to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream tas

In [103]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in tqdm(range(20)):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

100%|██████████| 20/20 [38:58<00:00, 116.95s/it]


CamembertForTokenClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0): CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
        

In [102]:
torch.save(model, 'model_v1.pb')

## Evaluation

In [18]:
model = torch.load('model_v1.pb')

input_ids, attention_masks = val_dataset.encodings['input_ids'], val_dataset.encodings['attention_mask']
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
flat_pred = []
with torch.no_grad():
    input_ids, attention_masks = torch.tensor(input_ids), torch.tensor(attention_masks)

    outputs =  model(input_ids.to(device), token_type_ids=None, attention_mask=attention_masks.to(device))

logits = outputs['logits'].detach().cpu().numpy()
pred = np.argmax(logits, axis=2).flatten()
true_labels = np.array(val_labels).flatten()

liste_results = []
for i in range(len(pred)):
    if id2tag[pred[i]] != 'O' or (true_labels[i] not in [-100, 0]):
        liste_results.append(pred[i]==true_labels[i])
print( "Sur l'ensemble des vrais labels différents de O et des labels prédits différents de O, "+"{:.1f}".format(np.mean(liste_results)*100)+"% ont le bon label prédit")

Sur l'ensemble des vrais labels différents de O et des labels prédits différents de O, 29.5% ont le bon label prédit
